- Select dataset
- Perform data preparation and cleaning using Pandas & Numpy
- Perform exploratory analysis & visualization using Matplotlib and Seaborn
- Ask and Answer questions about the data in a Jupyter Notebook
- Summarize your inference and write a conclusion
- Document, publish and present your Jupyter Notebook Online
- Create interactive Dashboard 

# American Values Data Analysis

TODO- talk about EDA

TODO- talk about the dataset (source, what it contains, how it will be useful)

- Source: Five Thirty Eight
- Can be useful to understand American values how 

# Data Preparation and Cleaning

- Load the file using pandas
- Look at some information about the file about the data and columns
- Fix any missing or incorrect values


## Load the file using pandas

In [34]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
df = pd.read_csv('nonvoters_data.csv')


df.head()

,RespId,weight,Q1,Q2_1,Q2_2,Q2_3,Q2_4,Q2_5,Q2_6,Q2_7,...,Q30,Q31,Q32,Q33,ppage,educ,race,gender,income_cat,voter_category
0,470001,0.7516,1,1,1,2,4,1,4,2,...,2,NaN,1.0,NaN,73,College,White,Female,$75-125k,always
1,470002,1.0267,1,1,2,2,3,1,1,2,...,3,NaN,NaN,1.0,90,College,White,Female,$125k or more,always
2,470003,1.0844,1,1,1,2,2,1,1,2,...,2,NaN,2.0,NaN,53,College,White,Male,$125k or more,sporadic
3,470007,0.6817,1,1,1,1,3,1,1,1,...,2,NaN,1.0,NaN,58,Some college,Black,Female,$40-75k,sporadic
4,480008,0.9910,1,1,1,-1,1,1,1,1,...,1,-1.0,NaN,NaN,81,High school or less,White,Male,$40-75k,always


## Look at information about the data and columns

In [42]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5836 entries, 0 to 5835
Columns: 119 entries, RespId to voter_category
dtypes: float64(23), int64(91), object(5)
memory usage: 5.3+ MB


By looking at the info from our dataframe, we learn that we have 119 columns in our dataset and 5,836 rows. 

There's too many columns to see the full number and so, to find out information about the columns, [we refer to the codebook for the dataset](https://github.com/fivethirtyeight/data/blob/master/non-voters/nonvoters_codebook.pdf).

This analysis will focus on questions two and three in the study and so we'll filter the dataframe to only include the pertinent columns.


In [43]:
df.describe()

,RespId,weight,Q1,Q2_1,Q2_2,Q2_3,Q2_4,Q2_5,Q2_6,Q2_7,...,Q29_6,Q29_7,Q29_8,Q29_9,Q29_10,Q30,Q31,Q32,Q33,ppage
count,5836.000000,5836.000000,5836.0,5836.000000,5836.000000,5836.000000,5836.000000,5836.000000,5836.000000,5836.000000,...,1342.000000,1342.000000,1342.000000,1342.000000,1342.000000,5836.000000,1592.000000,2002.000000,2242.000000,5836.000000
mean,474653.997772,0.991023,1.0,1.246402,1.705106,1.638280,2.175977,1.277245,1.805517,1.491604,...,-0.926975,-0.758569,-0.697466,-0.810730,-0.700447,2.325051,1.364950,1.365634,1.220339,51.693797
std,3628.475677,0.345022,0.0,0.660253,0.866346,0.765741,1.091391,0.626386,1.011524,0.808120,...,0.375264,0.651835,0.716885,0.585638,0.713970,1.259642,0.519249,0.497046,0.958569,17.071561
min,470001.000000,0.229800,1.0,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,...,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,22.000000
25%,472069.750000,0.793150,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,1.000000,1.000000,1.000000,1.000000,36.000000
50%,474152.000000,0.967600,1.0,1.000000,2.000000,2.000000,2.000000,1.000000,1.000000,1.000000,...,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,2.000000,1.000000,1.000000,1.000000,54.000000
75%,476217.500000,1.169600,1.0,1.000000,2.000000,2.000000,3.000000,1.000000,2.000000,2.000000,...,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,3.000000,2.000000,2.000000,2.000000,65.000000
max,488325.000000,3.038600,1.0,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,5.000000,2.000000,2.000000,2.000000,94.000000


Here, we notice that there's values of -1 for certain questions. We'll remove this since these don't have any value.


In [44]:
# Create dictionary with corresponding keys (we'll use later to rename the columns)

column_dict = {"Q2_1":"Voting",
        "Q2_2": "Jury",
        "Q2_3": "Govt_Politics",
        "Q2_4": "American_Flag",
        "Q2_5": "US_Census",
        "Q2_6": "Pledge",
        "Q2_7":"Military Support",
        "Q2_8": "Respecting_Others",
        "Q2_9":"Believing in God",
        "Q2_10":"Protesting", 
        "Q3_1": "Racism_US",
        "Q3_2": "Racism_Policing",
        "Q3_3":"Society_Whole",
        "Q3_4":"Mainstream_Media",
        "Q3_5":"Traditional_Parties",
        "Q3_5":"Way_People_Talk"}

# Create list of original column names 
original_column_names = list(column_dict.keys())

# Create two lists that we'll use to restructure the survey data
question_two = []
question_three = []
for key in original_column_names:
    if "Q2" in key:
        question_two.append(key)
    else:
        question_three.append(key)


In [45]:
# Remove values that are below zero 
for name in original_column_names:
    df.drop(df[df[name]<0].index, inplace = True)

In [46]:
df.describe()

,RespId,weight,Q1,Q2_1,Q2_2,Q2_3,Q2_4,Q2_5,Q2_6,Q2_7,...,Q29_6,Q29_7,Q29_8,Q29_9,Q29_10,Q30,Q31,Q32,Q33,ppage
count,5576.000000,5576.000000,5576.0,5576.000000,5576.000000,5576.000000,5576.000000,5576.000000,5576.000000,5576.000000,...,1264.000000,1264.000000,1264.000000,1264.000000,1264.000000,5576.000000,1530.000000,1913.000000,2133.000000,5576.000000
mean,474658.388451,0.990883,1.0,1.256636,1.717001,1.651901,2.203192,1.286047,1.825502,1.510760,...,-0.930380,-0.754747,-0.689873,-0.813291,-0.710443,2.333034,1.375163,1.370099,1.268636,51.489957
std,3615.104380,0.344344,0.0,0.632421,0.839328,0.737143,1.067143,0.605024,0.995345,0.778103,...,0.366742,0.656276,0.724217,0.582087,0.704033,1.244652,0.508049,0.492607,0.912461,17.029499
min,470001.000000,0.229800,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,22.000000
25%,472083.500000,0.794500,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,1.000000,1.000000,1.000000,1.000000,36.000000
50%,474179.500000,0.967600,1.0,1.000000,2.000000,2.000000,2.000000,1.000000,1.000000,1.000000,...,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,2.000000,1.000000,1.000000,1.000000,54.000000
75%,476231.250000,1.169675,1.0,1.000000,2.000000,2.000000,3.000000,1.000000,2.000000,2.000000,...,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,3.000000,2.000000,2.000000,2.000000,65.000000
max,488325.000000,3.038600,1.0,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,5.000000,2.000000,2.000000,2.000000,94.000000


2. In your view, how important are each of the following to being a good American?

   1. Voting in elections
   2. Serving on a jury
   3. Following what happens in government and politics
   4. Displaying the American flag
   5. Participating in the U.S. Census every ten years
   6. Knowing the Pledge of Allegiance
   7. Supporting the military
   8. Respecting the opinions of those who disagree with you
   9. Believing in God
   10. Protesting if you believe government actions are wrong
    
Responses
1. Very important
2. Somewhat important
3. Not so important
4. Not at all important


3. How much do you agree or disagree with the following statements?

   1. Systemic racism is a problem in the United States.
   2. Systemic racism in policing is a bigger problem than violence and vandalism
   in protests.
   1. Society as a whole has become too soft and feminine.
   2. The mainstream media is more interested in making money than telling the truth.
   3. Traditional parties and politicians don’t care about people like me.
   4. The way people talk needs to change with the times to be more sensitive to people with different backgrounds.

Responses
1. Strongly agree
2. Somewhat agree
3. Somewhat disagree
4. Strongly disagree

In [47]:
# Create list of original column names
column_list = list(df.columns)

question_two_columns = []
question_three_columns = []

for key in column_list:
    if "Q2_" in key:
        question_two_columns.append(key)
    elif "Q3_" in key:
        question_three_columns.append(key)


# Use Lambda function to relabel Likert scale data



In [33]:
def likert_converter (column_name):
    """Converet Likert Scale data"""
    if 'Q2_' in column_name:
        df[column_name+'_new'] = df[column_name].map(lambda x: 'Very Important' if x==1 
        else ('Somewhat important' if x==2 
        else('Not so important'if x==3 
        else 'Not at all important')))
    else:
        df[column_name+'_new'] = df[column_name].map(lambda x: 'Very Important' if x==1 
        else ('Somewhat important' 
        if x==2 else('Not so important'if x==3 
        else 'Not at all important')))

column_list = ['Q2_1','Q3_2']

for column in column_list:
    likert_converter(column)

df[['Q3_1_new','Q3_1','Q2_1_new']]

plt.figure(figsize=(8,5))
plot_title = ("Distribution of "+variable)
plt.title(plot_title)
sns.countplot(x=variable, data = dataframe, palette='rainbow')


,Q3_1_new,Q3_1,Q2_1_new
0,Very Important,1,Very Important
1,Not so important,3,Very Important
2,Somewhat important,2,Very Important
3,Very Important,1,Very Important
4,Not at all important,4,Very Important
...,...,...,...
5831,Very Important,1,Very Important
5832,Very Important,1,Very Important
5833,Very Important,1,Very Important
5834,Not at all important,4,Somewhat important


In [48]:
# Restructure Likert data for question two

question_two_new_values = ["Somewhat important","Not so important","Not at all important"]

# Lambda functions
# Encoding with Pandas 

# Question one

column_name = "Q2_1"



conditions = [
      np.logical_and(df[column_name].gt(1), np.less_equal(df[column_name], 2)),
      np.logical_and(df[column_name].gt(2), np.less_equal(df[column_name], 3)),
      np.logical_and(df[column_name].gt(3), np.less_equal(df[column_name], 4)),]

df[column_name] = pd.Series(np.select(conditions, question_two_new_values, "Very important"))

# Question two

column_name = "Q2_2"

conditions = [
      np.logical_and(df[column_name].gt(1), np.less_equal(df[column_name], 2)),
      np.logical_and(df[column_name].gt(2), np.less_equal(df[column_name], 3)),
      np.logical_and(df[column_name].gt(3), np.less_equal(df[column_name], 4)),]

df[column_name] = pd.Series(np.select(conditions, question_two_new_values, "Very important"))

# Question three

column_name = "Q2_3"


conditions = [
      np.logical_and(df[column_name].gt(1), np.less_equal(df[column_name], 2)),
      np.logical_and(df[column_name].gt(2), np.less_equal(df[column_name], 3)),
      np.logical_and(df[column_name].gt(3), np.less_equal(df[column_name], 4)),]

df[column_name] = pd.Series(np.select(conditions, question_two_new_values, "Very important"))

# Question four

column_name = "Q2_4"


conditions = [
      np.logical_and(df[column_name].gt(1), np.less_equal(df[column_name], 2)),
      np.logical_and(df[column_name].gt(2), np.less_equal(df[column_name], 3)),
      np.logical_and(df[column_name].gt(3), np.less_equal(df[column_name], 4)),]

df[column_name] = pd.Series(np.select(conditions, question_two_new_values, "Very important"))

# Question five

column_name = "Q2_5"


conditions = [
      np.logical_and(df[column_name].gt(1), np.less_equal(df[column_name], 2)),
      np.logical_and(df[column_name].gt(2), np.less_equal(df[column_name], 3)),
      np.logical_and(df[column_name].gt(3), np.less_equal(df[column_name], 4)),]

df[column_name] = pd.Series(np.select(conditions, question_two_new_values, "Very important"))

# Question six

column_name = "Q2_6"


conditions = [
      np.logical_and(df[column_name].gt(1), np.less_equal(df[column_name], 2)),
      np.logical_and(df[column_name].gt(2), np.less_equal(df[column_name], 3)),
      np.logical_and(df[column_name].gt(3), np.less_equal(df[column_name], 4)),]

df[column_name] = pd.Series(np.select(conditions, question_two_new_values, "Very important"))

# Question seven

column_name = "Q2_7"


conditions = [
      np.logical_and(df[column_name].gt(1), np.less_equal(df[column_name], 2)),
      np.logical_and(df[column_name].gt(2), np.less_equal(df[column_name], 3)),
      np.logical_and(df[column_name].gt(3), np.less_equal(df[column_name], 4)),]

df[column_name] = pd.Series(np.select(conditions, question_two_new_values, "Very important"))

# Question eight

column_name = "Q2_8"


conditions = [
      np.logical_and(df[column_name].gt(1), np.less_equal(df[column_name], 2)),
      np.logical_and(df[column_name].gt(2), np.less_equal(df[column_name], 3)),
      np.logical_and(df[column_name].gt(3), np.less_equal(df[column_name], 4)),]

df[column_name] = pd.Series(np.select(conditions, question_two_new_values, "Very important"))

# Question nine

column_name = "Q2_9"


conditions = [
      np.logical_and(df[column_name].gt(1), np.less_equal(df[column_name], 2)),
      np.logical_and(df[column_name].gt(2), np.less_equal(df[column_name], 3)),
      np.logical_and(df[column_name].gt(3), np.less_equal(df[column_name], 4)),]

df[column_name] = pd.Series(np.select(conditions, question_two_new_values, "Very important"))

# Question ten

column_name = "Q2_10"


conditions = [
      np.logical_and(df[column_name].gt(1), np.less_equal(df[column_name], 2)),
      np.logical_and(df[column_name].gt(2), np.less_equal(df[column_name], 3)),
      np.logical_and(df[column_name].gt(3), np.less_equal(df[column_name], 4)),]

df[column_name] = pd.Series(np.select(conditions, question_two_new_values, "Very important"))


In [49]:
column_dict = {"Q2_1":"Voting",
        "Q2_2": "Jury",
        "Q2_3": "Govt_Politics",
        "Q2_4": "American_Flag",
        "Q2_5": "US_Census",
        "Q2_6": "Pledge",
        "Q2_7":"Military Support",
        "Q2_8": "Respecting_Others",
        "Q2_9":"Believing in God",
        "Q2_10":"Protesting"}


df = df.rename(columns=column_dict)

df


,RespId,weight,Q1,Voting,Jury,Govt_Politics,American_Flag,US_Census,Pledge,Military Support,...,Q30,Q31,Q32,Q33,ppage,educ,race,gender,income_cat,voter_category
0,470001,0.7516,1,Very important,Very important,Somewhat important,Not at all important,Very important,Not at all important,Somewhat important,...,2,NaN,1.0,NaN,73,College,White,Female,$75-125k,always
1,470002,1.0267,1,Very important,Somewhat important,Somewhat important,Not so important,Very important,Very important,Somewhat important,...,3,NaN,NaN,1.0,90,College,White,Female,$125k or more,always
2,470003,1.0844,1,Very important,Very important,Somewhat important,Somewhat important,Very important,Very important,Somewhat important,...,2,NaN,2.0,NaN,53,College,White,Male,$125k or more,sporadic
3,470007,0.6817,1,Very important,Very important,Very important,Not so important,Very important,Very important,Very important,...,2,NaN,1.0,NaN,58,Some college,Black,Female,$40-75k,sporadic
6,480010,1.1512,1,Very important,Very important,Very important,Not so important,Very important,Very important,Very important,...,1,1.0,NaN,NaN,80,High school or less,White,Female,$125k or more,always
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5831,477662,1.1916,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2,NaN,1.0,NaN,27,Some college,Hispanic,Male,$40-75k,always
5832,477663,1.4623,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2,NaN,2.0,NaN,59,High school or less,White,Female,$125k or more,rarely/never
5833,488322,0.9252,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2,NaN,1.0,NaN,51,College,Other/Mixed,Male,$125k or more,sporadic
5834,488325,2.6311,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3,NaN,NaN,1.0,22,High school or less,Black,Female,Less than $40k,always


In [50]:
# Restructure Likert data for question three

question_three_new_values = ["Somewhat agree", "Somewhat disagree", "Strongly disagree"]

# Question one

column_name = "Q3_1"

conditions = [
      np.logical_and(df[column_name].gt(1), np.less_equal(df[column_name], 2)),
      np.logical_and(df[column_name].gt(2), np.less_equal(df[column_name], 3)),
      np.logical_and(df[column_name].gt(3), np.less_equal(df[column_name], 4)),]

df[column_name] = pd.Series(np.select(conditions, question_two_new_values, "Strong agree"))

# Question two

column_name = "Q3_2"

conditions = [
      np.logical_and(df[column_name].gt(1), np.less_equal(df[column_name], 2)),
      np.logical_and(df[column_name].gt(2), np.less_equal(df[column_name], 3)),
      np.logical_and(df[column_name].gt(3), np.less_equal(df[column_name], 4)),]

df[column_name] = pd.Series(np.select(conditions, question_two_new_values, "Strong agree"))

# Question three

column_name = "Q3_3"

conditions = [
      np.logical_and(df[column_name].gt(1), np.less_equal(df[column_name], 2)),
      np.logical_and(df[column_name].gt(2), np.less_equal(df[column_name], 3)),
      np.logical_and(df[column_name].gt(3), np.less_equal(df[column_name], 4)),]

df[column_name] = pd.Series(np.select(conditions, question_two_new_values, "Strong agree"))

# Question four

column_name = "Q3_4"

conditions = [
      np.logical_and(df[column_name].gt(1), np.less_equal(df[column_name], 2)),
      np.logical_and(df[column_name].gt(2), np.less_equal(df[column_name], 3)),
      np.logical_and(df[column_name].gt(3), np.less_equal(df[column_name], 4)),]

df[column_name] = pd.Series(np.select(conditions, question_two_new_values, "Strong agree"))

# Question five

column_name = "Q3_5"

conditions = [
      np.logical_and(df[column_name].gt(1), np.less_equal(df[column_name], 2)),
      np.logical_and(df[column_name].gt(2), np.less_equal(df[column_name], 3)),
      np.logical_and(df[column_name].gt(3), np.less_equal(df[column_name], 4)),]

df[column_name] = pd.Series(np.select(conditions, question_two_new_values, "Strong agree"))


In [51]:
# Rename the columns

column_dict = {"Q2_1":"Voting",
        "Q2_2": "Jury",
        "Q2_3": "Govt_Politics",
        "Q2_4": "American_Flag",
        "Q2_5": "US_Census",
        "Q2_6": "Pledge",
        "Q2_7":"Military Support",
        "Q2_8": "Respecting_Others",
        "Q2_9":"Believing in God",
        "Q2_10":"Protesting", 
        "Q3_1": "Racism_US",
        "Q3_2": "Racism_Policing",
        "Q3_3":"Society_Whole",
        "Q3_4":"Mainstream_Media",
        "Q3_5":"Traditional_Parties",
        "Q3_5":"Way_People_Talk"}


df = df.rename(columns=column_dict)


In [52]:
# Create a subset of our data


df = df.copy().loc[:,["Voting",
                "Jury",
                "Govt_Politics",
                "American_Flag",
                "US_Census",
                "Pledge",
                "Military Support",
                "Respecting_Others",
                "Believing in God",
                "Protesting",
                "Racism_US",
                "Racism_Policing",
                "Society_Whole",
                "Mainstream_Media",
                "Way_People_Talk",
                "ppage",
                "educ",
                "race",
                "gender",
                "income_cat",
                "voter_category"]]

In [53]:
# Checking for missing values for our dataframe
df.isna().sum().sort_values()

voter_category         0
gender                 0
race                   0
educ                   0
ppage                  0
income_cat             0
Way_People_Talk      250
Mainstream_Media     250
Society_Whole        250
Racism_Policing      250
Voting               250
Believing in God     250
Respecting_Others    250
Military Support     250
Pledge               250
US_Census            250
American_Flag        250
Govt_Politics        250
Jury                 250
Protesting           250
Racism_US            250
dtype: int64

In [61]:
df.dropna()

,Voting,Jury,Govt_Politics,American_Flag,US_Census,Pledge,Military Support,Respecting_Others,Believing in God,Protesting,...,Society_Whole,Mainstream_Media,Way_People_Talk,ppage,educ,race,gender,income_cat,voter_category,age_binned
0,Very important,Very important,Somewhat important,Not at all important,Very important,Not at all important,Somewhat important,Somewhat important,Not at all important,Somewhat important,...,Not at all important,Not at all important,Not so important,73,College,White,Female,$75-125k,always,66-94
1,Very important,Somewhat important,Somewhat important,Not so important,Very important,Very important,Somewhat important,Very important,Very important,Not so important,...,Not at all important,Not so important,Not so important,90,College,White,Female,$125k or more,always,66-94
2,Very important,Very important,Somewhat important,Somewhat important,Very important,Very important,Somewhat important,Very important,Not at all important,Not so important,...,Not so important,Not so important,Somewhat important,53,College,White,Male,$125k or more,sporadic,37-54
3,Very important,Very important,Very important,Not so important,Very important,Very important,Very important,Very important,Very important,Somewhat important,...,Not at all important,Not at all important,Somewhat important,58,Some college,Black,Female,$40-75k,sporadic,55-65
6,Very important,Very important,Very important,Not so important,Very important,Very important,Very important,Very important,Not so important,Somewhat important,...,Somewhat important,Strong agree,Somewhat important,80,High school or less,White,Female,$125k or more,always,66-94
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5571,Very important,Not so important,Very important,Not so important,Very important,Somewhat important,Somewhat important,Somewhat important,Not so important,Very important,...,Not so important,Strong agree,Strong agree,53,High school or less,White,Male,$40-75k,always,37-54
5572,Very important,Very important,Very important,Somewhat important,Very important,Somewhat important,Very important,Very important,Very important,Very important,...,Not so important,Somewhat important,Somewhat important,29,Some college,White,Female,$40-75k,always,22-36
5573,Very important,Somewhat important,Very important,Not so important,Very important,Very important,Somewhat important,Very important,Not at all important,Very important,...,Not at all important,Somewhat important,Not at all important,65,College,White,Female,$125k or more,sporadic,55-65
5574,Somewhat important,Somewhat important,Somewhat important,Somewhat important,Somewhat important,Somewhat important,Somewhat important,Somewhat important,Somewhat important,Somewhat important,...,Not so important,Not so important,Not so important,22,Some college,White,Female,Less than $40k,always,22-36


In [62]:
df.isna().sum().sort_values()

age_binned             0
income_cat             0
gender                 0
race                   0
educ                   0
ppage                  0
voter_category         0
Way_People_Talk      250
Mainstream_Media     250
Society_Whole        250
Racism_Policing      250
Voting               250
Believing in God     250
Respecting_Others    250
Military Support     250
Pledge               250
US_Census            250
American_Flag        250
Govt_Politics        250
Jury                 250
Protesting           250
Racism_US            250
dtype: int64

In [56]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5576 entries, 0 to 5835
Data columns (total 21 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Voting             5326 non-null   object
 1   Jury               5326 non-null   object
 2   Govt_Politics      5326 non-null   object
 3   American_Flag      5326 non-null   object
 4   US_Census          5326 non-null   object
 5   Pledge             5326 non-null   object
 6   Military Support   5326 non-null   object
 7   Respecting_Others  5326 non-null   object
 8   Believing in God   5326 non-null   object
 9   Protesting         5326 non-null   object
 10  Racism_US          5326 non-null   object
 11  Racism_Policing    5326 non-null   object
 12  Society_Whole      5326 non-null   object
 13  Mainstream_Media   5326 non-null   object
 14  Way_People_Talk    5326 non-null   object
 15  ppage              5576 non-null   int64 
 16  educ               5576 non-null   object


We can see that we have fifteen questions for our dataset (now restructured as string data). 

The other six columns represent socioeconomic data points. Aside from age, these are all object data types (or categorical data).


Last but not least, we'll bin our age data so that it's easier to visualize.

In [57]:
df["ppage"].describe()

count    5576.000000
mean       51.489957
std        17.029499
min        22.000000
25%        36.000000
50%        54.000000
75%        65.000000
max        94.000000
Name: ppage, dtype: float64

In [58]:
# Use quartile values to create bins

age_bins = ["22-36","37-54","55-65","66-94"]

df['age_binned'] = pd.qcut(df['ppage'],
                              q=[0,0.25, 0.5, 0.75, 1],
                              labels=age_bins)


## Fix any missing or incorrect values


In [59]:
# Create list of survey data columns - need to use to show that there's data types that will need to be restructured

survey_data_columns = []
for column in df.columns:
    if df[column].dtypes == "int64" and column != 'ppage':
        survey_data_columns.append(column)

# Perform exploratory analysis & visualization using Matplotlib and Seaborn

In [60]:
# Create visualizations for dataset 

# Ask and Answer questions about the data in a Jupyter Notebook

- Do people of certain backgrounds tend to think differently American values?
- Do certain answers correlate with each other?

# Summarize your inference and write a conclusion
